#### `QR algorithm`

So far, we iterate based on

$$Q_{i+1}R_{i+1}=AQ_i$$

We can rearrange to do it slightly differently

$$Q_i^TQ_{i+1}R_{i+1}=Q_i^TAQ_i$$

If we denote $\tilde{Q}_{i+1}=Q_i^TQ_{i+1}$, and $Q_i^TAQ_i=A_i$, then, by construction, $\tilde{Q}_{i+1}$ is `orthogonal`, and $A_i$ is `similar` to $A$ and therefore, have the same eigenvalues

So we have

$$\tilde{Q}_{i+1}R_{i+1}=A_i$$

Now if we swap the factors, we get

$$\begin{align*}
R_{i+1}\tilde{Q}_{i+1}&=\left(Q^T_{i+1}AQ_i\right)\left(Q_i^TQ_{i+1}\right) \\
&=Q^T_{i+1}AQ_{i+1} \\
&=A_{i+1}
\end{align*}$$

So far, we haven't really changed anything to orthogonal iterations, just reformulate it at each step by regrouping and reusing the outcomes from orthogonal iterations

Therefore, if orthogonal iterations converge as $i\rightarrow \infty$, then

$$Q^TAQ=A_{i+1}$$

and $T$ in Schur decomposition is $A_{i+1}$

This indicates that instead of iterating over $Q$ as in orthogonal iterations, equivalently, we can also iterate over $T$ (in this case, $A$ itself)

This is the basic `QR algorithm` for finding eigenvalues of general matrices

(In practice, many other tricks would be needed, as usual...)

In [4]:
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(formatter={'float': '{: 0.4f}'.format})

plt.style.use('dark_background')
# color: https://matplotlib.org/stable/gallery/color/named_colors.htm

In [5]:
def householder(A):
    m, n = A.shape
    R = A.copy()
    Q = np.identity(m)

    for i in range(n):
        x = R[i:, i]
        v = np.sign(x[0]) * np.linalg.norm(x) * np.eye(x.shape[0])[:,0] + x
        v /= np.linalg.norm(v)

        # Since all entries in R[i:, :i] are zero from previous iteration
        # applying transformation to R[i:, i:] would suffice
        R[i:, i:] -= 2 * np.outer(v, v) @ R[i:, i:]

        # If Q is needed explicitly
        Q[i:, :] -= 2 * np.outer(v, v) @ Q[i:, :]

    return Q.T, R

def diagonalizable_mat(n):
    # Create diagonal matrix D with eigenvalues
    D = np.diag(np.concatenate((200*np.random.rand(n//2)-100, 0.1*np.random.rand(n-n//2))))

    # Generate a random invertible matrix
    P = np.random.rand(n, n)
    while np.linalg.cond(P) > 1e8:  # Check conditioning
        P = np.random.rand(n, n)

    # Use similarity transformation to create diagonalizable, but nonsymmetric matrix
    return P @ D @ np.linalg.inv(P)

In [6]:
np.random.seed(50)

A_size = 8
A = diagonalizable_mat(A_size)
A_original = A.copy()
Q = np.eye(A.shape[0])

num_iter = 51

# QR algorithm
for i in range(num_iter):
    Q, R = householder(A)
    A = R @ Q

    # Diagonal elements of A are approximation of eigenvalues
    if i % 10 == 0:
        print(np.diag(A))

# Compare to NumPy
eigenvalues, _ = np.linalg.eig(A_original)
print(f'\nEigenvalues from NumPy: \n{eigenvalues}')

[-42.0239 -41.1004 -42.6385  0.8502 -0.0361  0.0942  0.0068  0.0009]
[-54.6554 -48.6274 -20.7398 -1.0797  0.0996  0.0772  0.0403  0.0383]
[-54.4753 -48.8133 -20.7340 -1.0797  0.0997  0.0772  0.0406  0.0380]
[-54.4149 -48.8737 -20.7340 -1.0797  0.0997  0.0772  0.0407  0.0378]
[-54.3942 -48.8944 -20.7340 -1.0797  0.0997  0.0772  0.0408  0.0378]
[-54.3871 -48.9015 -20.7340 -1.0797  0.0997  0.0772  0.0408  0.0377]

Eigenvalues from NumPy: 
[-54.3834 -48.9052 -20.7340 -1.0797  0.0997  0.0377  0.0408  0.0772]
